In [2]:
import sys
import os
sys.path.insert(0, '../driver')

In [3]:
import numpy as np
import math
import plotly.graph_objects as go
from control import move_axis_data, MoveAxisRateInterpolator

In [ ]:
def plot_slow_interpolation(axis_id):
    axis_data = move_axis_data[axis_id]
    interpolator = MoveAxisRateInterpolator(axis_data).ASCOM

    idx = axis_data["RAW"].index(5) + 1
    ascom_rates = np.linspace(axis_data["ASCOM"][0], axis_data["ASCOM"][idx - 1], 100)
    interpolated_raw = interpolator.SLOW(ascom_rates)
    interpolated_dps = MoveAxisRateInterpolator(axis_data).RAW.toDPS(interpolated_raw)

    fig = go.Figure()

    # Interpolated line
    fig.add_trace(go.Scatter(
        x=ascom_rates,
        y=interpolated_dps,
        mode='lines',
        name='Interpolated DPS (Slow)',
        line=dict(color='blue')
    ))

    # Original data points
    fig.add_trace(go.Scatter(
        x=axis_data["ASCOM"][:idx],
        y=axis_data["DPS"][:idx],
        mode='markers',
        name='Data Points',
        marker=dict(color='blue', size=6)
    ))

    fig.update_layout(
        title=f"Axis {axis_id} — ASCOM Rate vs DPS (Slow Interpolation)",
        xaxis_title="ASCOM Rate",
        yaxis_title="Degrees Per Second (DPS)",
        template="plotly_white"
    )

    fig.show()

def plot_fast_interpolation(axis_id):
    axis_data = move_axis_data[axis_id]
    interpolator_raw = MoveAxisRateInterpolator(axis_data).RAW
    interpolator_ascom = MoveAxisRateInterpolator(axis_data).ASCOM

    idx = axis_data["RAW"].index(5) + 1

    # RAW interpolation
    raw_rates = np.linspace(axis_data["RAW"][idx], axis_data["RAW"][-1], 100)
    dps_from_raw = interpolator_raw.toDPS(raw_rates)

    # ASCOM interpolation (ASCOM → RAW → DPS)
    ascom_rates = np.linspace(axis_data["ASCOM"][idx], axis_data["ASCOM"][-1], 100)
    raw_from_ascom = interpolator_ascom.toRAW(ascom_rates)
    dps_from_ascom = interpolator_raw.toDPS(raw_from_ascom)

    fig = go.Figure()

    # Interpolated ASCOM → DPS line
    fig.add_trace(go.Scatter(
        x=ascom_rates,
        y=dps_from_ascom,
        mode='lines',
        name='Interpolated ASCOM → DPS',
        line=dict(color='green')
    ))

    # Interpolated RAW → DPS line
    fig.add_trace(go.Scatter(
        x=raw_rates,
        y=dps_from_raw,
        mode='lines',
        name='Interpolated RAW → DPS',
        line=dict(color='red', dash='dot')
    ))

    # Original points
    fig.add_trace(go.Scatter(
        x=axis_data["ASCOM"][idx:],
        y=axis_data["DPS"][idx:],
        mode='markers',
        name='ASCOM Data Points',
        marker=dict(color='green', size=6)
    ))

    fig.add_trace(go.Scatter(
        x=axis_data["RAW"][idx:],
        y=axis_data["DPS"][idx:],
        mode='markers',
        name='RAW Data Points',
        marker=dict(color='red', size=6)
    ))

    fig.update_layout(
        title=f"Axis {axis_id} — ASCOM + RAW Rate vs DPS (Fast Interpolation)",
        xaxis_title="Rate",
        yaxis_title="Degrees Per Second (DPS)",
        template="plotly_white"
    )

    fig.show()


In [6]:
plot_slow_interpolation(0)

In [ ]:
def plot_polar_rotation_angles(latitude_deg, location_name):
    # Generate grid of azimuth and altitude values
    az_deg = np.linspace(0, 360, 360)
    alt_deg = np.linspace(-90, 90, 200)
    AZ, ALT = np.meshgrid(az_deg, alt_deg)

    # Convert to radians
    az_rad = np.radians(AZ)
    alt_rad = np.radians(ALT)
    latitude_rad = math.radians(latitude_deg)

    # Compute rotation angles
    rotation_angles = np.vectorize(polar_rotation_angle)(latitude_rad, az_rad, alt_rad)

    # Create Plotly contour plot
    fig = go.Figure(data=go.Contour(
        z=rotation_angles,
        x=az_deg,
        y=alt_deg,
        colorscale='twilight',
        contours_coloring='heatmap',
        colorbar=dict(title='Polar Rotation Angle (°)'),
        hovertemplate='Azimuth: %{x:.1f}°<br>Altitude: %{y:.1f}°<br>Rotation: %{z:.2f}°<extra></extra>',
    ))

    fig.update_layout(
        title=f'{location_name}: Rotation Angle vs Azimuth and Altitude',
        xaxis_title='Azimuth (°)',
        yaxis_title='Altitude (°)',
        width=900,
        height=600,
    )

    fig.show()


In [ ]:


# Southern Hemisphere Test Location
plot_polar_rotation_angles(-33.85988, 'Southern Hemisphere (Sydney ≈ -33.85988°)')


In [ ]:
# Northern Hemisphere Test Location
plot_polar_rotation_angles(40.6972846, 'Northern Hemisphere (New York ≈ 40.6972846°)')
